Udemy Course Scraper

In [ ]:
# I worry that all of these xpath links will get broken by the react they're using. They had strange names.

# Change this variable if you're running all and want to overwrite existing cookies
force_new_cookies = False

# Parameters
cookies_path = "cookies.json"
purchase_page_pause = 20 # It can take a moment to load (increase or decrease this depending on your internet)

# Options for selenium
import undetected_chromedriver as uc
from undetected_chromedriver import By
from typing import Union, Tuple
import pandas as pd
import time
import json
import os
import datetime

# Items to store the purchases (would probably be better to do SQLite or DuckDB for persistence, but this will work for now)
purchases_df = pd.DataFrame(columns =["Date", "Purchase", "Price", "Receipt", "Invoice"])
courses_df = pd.DataFrame(columns =["Date", "Purchase", "Price", "Receipt", "Invoice", "Course"])
purchases_list = []
courses_list = []

options = uc.ChromeOptions()
driver = uc.Chrome(use_subprocess=False, options=options)

# Go to udemy
driver.get("https://www.udemy.com/")

Manually log into Udemy now because trying to log in automatically with 2FA and everything will be a pain in the ass. Save the cookies to skip this step later.

I tried to export my cookies from Chrome and load them into here before, but it didn't work. So be it.

In [ ]:
# Checking if the cookies file already exists
if(force_new_cookies or not os.path.exists(cookies_path)):
    # Wait for an input for the user to log in
    input("Continue after you've logged into Udemy!")
    # To save cookies
    saved_cookies = driver.get_cookies()
    with open(cookies_path, "w") as file:
        json.dump(saved_cookies, file)

Load the Cookies in (don't need to do this if we just made them... but oh well)

In [ ]:
# To load in cookies
with open(cookies_path, "r") as file:
    saved_cookies = json.load(file)
for x in saved_cookies:
    driver.add_cookie(x)
# Wait for one second just to be sure (probably not needed)
time.sleep(1)

Functions

In [ ]:
# Function to check if there are multiple courses in a purchase
def check_multiple_courses(web_element: uc.WebElement) -> Union[bool, uc.WebElement]:
    '''
    Checks if a purchase is a single course or multiple courses

    :param web_element: The selenium web_element
    :type web_element: WebElement

    :return: Either False or the WebElement to the button to show more
    :rtype: bool or WebElement
    '''
    try:
        temp = web_element.find_element(By.XPATH, ".//button[@class='ud-btn ud-btn-medium ud-btn-link ud-heading-sm show-more_focusable-label__dmEhW']")
        return(temp)
    except:
        return(False)
# Function to check if a refund button exists for a course
def check_refund(web_element: uc.WebElement) -> bool:
    '''
    Checks if a course has a refund (inside of a larger purchase, not individual courses)

    :param web_element: The selenium web_element
    :type web_element: WebElement

    :return: True or False depending on if it has been refunded or not
    :rtype: bool

    '''
    try:
        # This will fail if the element can't be found
        web_element.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-details--text__JKVm6']")
        return(True)
    except:
        return(False)

Function for getting purchase information per page

In [ ]:
def scrape_purchase_page(t_purchases_list: list, t_courses_list: list, t_driver: uc.Chrome) -> Tuple[list, list]:
    '''
    The main function to scrape the individual purchase pages

    :param t_purchases_list: The list to store the purchases
    :type t_purchases_list: list

    :param t_courses_list: The list to store the courses
    :type t_courses_list: list

    :param t_driver: The web driver
    :type t_driver: Chrome

    :return: The lists you input but with the additional purchases and courses appended [t_purchases_list, t_courses_list]
    :rtype: Tuple[list, list]

    '''
    # Get the purchases
    purchase_table = t_driver.find_element(By.XPATH, "//table[@class='paginated-table_purchase-history-table__ncCll']")
    purchases = purchase_table.find_elements(By.XPATH, ".//tr[@class='paginated-table_purchase-history-table--main-row__5hHCb']")

    for ind, purchase in enumerate(purchases):
        # Get the purchase
        temp_purchase = purchase.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-details--text-container__EvFRz']")
        # Get the receipt
        receipt = purchase.find_element(By.XPATH, ".//span[@class='ud-btn-label' and contains(., 'Receipt')]")
        receipt = receipt.find_element(By.XPATH, "..")
        receipt = receipt.get_attribute("href")
        receipt = receipt.split("/")[-2]
        # Get the invoice (removing because not every purchase has an invoice and the receipt is what is useful anyway)
        # invoice = purchase.find_element(By.XPATH, ".//span[@class='ud-btn-label' and contains(., 'Invoice')]")
        # invoice = invoice.find_element(By.XPATH, "..")
        # invoice = invoice.get_attribute("href")
        # invoice = invoice.split("/")[-2]
        # Total price
        total_price = purchase.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-history-table--data-cell__LO_7L']/div[@class='app_positive-text___Wb07']").text
        # Get the data ( I couldn't also done find_elements and selected the index 1, but this should be better and was neat to learn)
        purchase_date = purchase.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-history-table--data-cell__LO_7L' and not(descendant::*)]").text
        
        # Add the purchase
        cur_purchase = {"Date": purchase_date, 
                        "Purchase": ind,
                        "Price": total_price,
                        "Receipt": receipt}
        t_purchases_list.append(cur_purchase)

        # Check if we need to check for a refund or not (over 30 days won't have the ability to return)
        check_refund_bool = False
        course_date = datetime.datetime.strptime(purchase_date, "%b %d, %Y").date()
        current_date = datetime.datetime.now().date()
        if((current_date - course_date).days <= 30):
            check_refund = True

        # Check if it has multiple purchases
        temp_multiple = check_multiple_courses(temp_purchase)
        if temp_multiple != False:
            # Click open the courses
            temp_multiple.click()
            # Wait for it to load because it can take a moment
            time.sleep(1)
            # Get the individual courses
            courses = purchase_table.find_elements(By.XPATH, ".//tr[@class='paginated-table_purchase-history-table--sub-row__2g87b']")
            for course in courses:
                # Check if it has a refund button
                if check_refund_bool:
                    course_refund = check_refund(course)
                else:
                    course_refund = True
                if course_refund:
                    course_name = course.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-details--text__JKVm6 paginated-table_purchase-details--content-title-text__DxDuf ud-text-md']")
                    course_name = course_name.find_element(By.XPATH, ".//a")
                    c_name = course_name.get_attribute("innerHTML")
                    c_link = course_name.get_attribute("href")
                    course_price = course.find_element(By.XPATH, ".//div[@class='app_positive-text___Wb07']").text
                    # Add the course
                    cur_course = {"Date": purchase_date, 
                                    "Purchase": ind,
                                    "Price": course_price,
                                    "Receipt": receipt,
                                    "Course": c_name,
                                    "Link": c_link}
                    t_courses_list.append(cur_course)
            # Close what we opened because the subrows are inserted directly into it
            temp_multiple.click()
        # If it's a singular course
        else:
            # Check for refund button
            try:
                if check_refund_bool:
                    course_refund = temp_purchase.find_element(By.XPATH, ".//div[@data-testid='redirect-button']")
                course_name = temp_purchase.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-details--text__JKVm6 paginated-table_purchase-details--content-title-text__DxDuf ud-text-md']")
                c_name = course_name.find_element(By.XPATH, ".//a").text
                c_link = course_name.get_attribute("href")
                # Add the course
                cur_course = {"Date": purchase_date, 
                                "Purchase": ind,
                                "Price": total_price,
                                "Receipt": receipt,
                                "Course": c_name,
                                "Link": c_link}
                t_courses_list.append(cur_course)
            except:
                pass
    return(t_purchases_list, t_courses_list)

Function to get the purchases

In [ ]:
purchase_page_pause = 10

def scrape_purchases(t_purchases_list: list, t_courses_list: list, t_driver: uc.Chrome) -> Tuple[list, list]:
    '''
    The function to handle multiple pages of purchases

    :param t_purchases_list: The list to store the purchases
    :type t_purchases_list: list

    :param t_courses_list: The list to store the courses
    :type t_courses_list: list

    :param t_driver: The web driver
    :type t_driver: Chrome

    :return: The lists you input but with the additional purchases and courses appended
    :rtype: Tuple[list, list]

    '''
    # Go to the purchases page initially
    t_driver.get("https://www.udemy.com/dashboard/purchase-history/")

    while True:
                # Wait for the page to load
                time.sleep(purchase_page_pause)
                # Scrape the page
                t_purchases_list, t_courses_list = scrape_purchase_page(t_purchases_list, t_courses_list, t_driver)
                # Get the next page button (it won't find it when we've done the last page)
                next_page = t_driver.find_element(By.XPATH, ".//a[@class='ud-btn ud-btn-medium ud-btn-secondary ud-btn-round ud-heading-sm ud-btn-icon ud-btn-icon-medium ud-btn-icon-round pagination_next__aBqfT'" \
                "and @aria-label='next page' and @aria-disabled='false']")
                next_page.click()

    try:
        # Inside a while loop because the amount of pages is unknown
        while True:
            # Wait for the page to load
            time.sleep(purchase_page_pause)
            # Scrape the page
            t_purchases_list, t_courses_list = scrape_purchase_page(t_purchases_list, t_courses_list, t_driver)
            # Get the next page button (it won't find it when we've done the last page)
            next_page = driver.find_element(By.XPATH, ".//a[@class='ud-btn ud-btn-medium ud-btn-secondary ud-btn-round ud-heading-sm ud-btn-icon ud-btn-icon-medium ud-btn-icon-round pagination_next__aBqfT'" \
            "and @aria-label='next page' and @aria-disabled='false']")
            next_page.click()
    # When the next page button isn't found return
    except:
        return(t_purchases_list, t_courses_list)
        
    

In [ ]:
scrape_purchases(purchases_list, courses_list, driver)
print(purchases_list)
print(courses_list)

In [ ]:
# Remove these after created
# Items to store the purchases (would probably be better to do SQLite or DuckDB for persistence, but this will work for now)
purchases_df = pd.DataFrame(columns =["Date", "Purchase", "Price", "Receipt", "Invoice"])
courses_df = pd.DataFrame(columns =["Date", "Purchase", "Price", "Receipt", "Invoice", "Course"])
purchases_list = []
courses_list = []
purchase_page_pause = 20 # It can take a moment to load (increase or decrease this depending on your internet)
